In [1]:
import sys
sys.path.append('..')
from src.utilities import *
from src.models.train_model import train_model
from src.models.param_opt import bayes_parameter_opt_lgb

## 3. Modelling

### 3.1 Load master table

In [2]:
master = pd.read_csv(os.path.join(processed_path, 'master.csv'))

### 3.2 Train / val / test split

In [3]:
x_train = master[(master.sales.isna()==False)].drop(columns = ['region', 'brand']).copy()
x_train = x_train[x_train.month >= '2020-06']
x_train['train'] =  (x_train.month <= '2021-06').astype(float)
x_train.drop(columns = 'month', inplace = True)

### 3.3 Bayesian parameter search

In [4]:
# best_params = bayes_parameter_opt_lgb(X = x_train.drop(columns = ['sales', 'train']), y = x_train.sales,
#                                       init_round=100, opt_round=30, n_folds=5, random_seed=6, n_estimators=10000, 
#                                       learning_rate=0.01, save_path = '', metric = 'mae').max['params']
# best_params

### 3.4 Model training

In [5]:
##### Define best parameters found earlier
best_params = {'metric': 'mae',
               'bagging_fraction': 0.8905860146003352,
               'feature_fraction': 0.2732587788087377,
               'lambda_l1': 3.5929572197000663,
               'lambda_l2': 0.05745304407908214,
               'max_depth': 17,
               'min_child_weight': 6.409030503082862,
               'min_split_gain': 0.012015118137189213,
               'num_leaves': 40}

##### Train a quantile regression lgb at different alpha levels
quantile_alphas = [0.1, 0.5, 0.9]
lgb_quantiles = {}

for alpha in quantile_alphas:
    current_model = train_model(x_train, target_name = 'sales', 
                                model_type = 'lgb-quantile', 
                                quantile_alpha = alpha,
                                params = best_params, 
                                metric = 'rmse', 
                                split = 'in_sample',
                                save_path = os.path.join(models_path, 
                                                         'model2_quantile_'+ str(alpha)+'.pkl'))
    lgb_quantiles[alpha] = current_model
    
##### Visualize feature contributions
feature_contributions = pd.DataFrame({'feature': x_train.drop(columns = ['sales', 'train']).columns, 
                                      'gain': lgb_quantiles[0.5].feature_importance(importance_type = 'gain'),
                                      'split': lgb_quantiles[0.5].feature_importance(importance_type = 'split')
                                     }).sort_values('gain', ascending = False)

feature_contributions[feature_contributions.gain>0][:40]



# 0.2 - 989.433  1047.01
# 0.5 - 496.95  672.037
# 0.8 -   690.059  923.275

####################     training with  4077      ####################
####################     validating with  453      ####################
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022599 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 32922
[LightGBM] [Info] Number of data points in the train set: 4077, number of used features: 479
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l1: 1792.89	valid_1's l1: 1729.57
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's l1: 1777.52	valid_1's l1: 1713.33
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	training's l1: 1761.34	valid_1's l1: 1697.16
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	training's l1: 1745.55	valid_1's l1: 1680.72
[LightG

[159]	training's l1: 1169.21	valid_1's l1: 1170.19
[160]	training's l1: 1169.06	valid_1's l1: 1170.08
[161]	training's l1: 1167.78	valid_1's l1: 1169.65
[162]	training's l1: 1167.34	valid_1's l1: 1169.45
[163]	training's l1: 1166.59	valid_1's l1: 1168.95
[164]	training's l1: 1165.44	valid_1's l1: 1168.25
[165]	training's l1: 1165.07	valid_1's l1: 1168.05
[166]	training's l1: 1163.34	valid_1's l1: 1166.43
[167]	training's l1: 1162.91	valid_1's l1: 1166.13
[168]	training's l1: 1162.81	valid_1's l1: 1166.05
[169]	training's l1: 1162.7	valid_1's l1: 1166.02
[170]	training's l1: 1161.79	valid_1's l1: 1165.35
[171]	training's l1: 1161.41	valid_1's l1: 1165.05
[172]	training's l1: 1160.04	valid_1's l1: 1164.07
[173]	training's l1: 1158.98	valid_1's l1: 1163.35
[174]	training's l1: 1157.37	valid_1's l1: 1161.89
[175]	training's l1: 1156.5	valid_1's l1: 1161.12
[176]	training's l1: 1154.56	valid_1's l1: 1159.8
[177]	training's l1: 1153.25	valid_1's l1: 1158.92
[178]	training's l1: 1152.34	valid

[331]	training's l1: 1103.47	valid_1's l1: 1122.64
[332]	training's l1: 1103.4	valid_1's l1: 1122.59
[333]	training's l1: 1103.33	valid_1's l1: 1122.5
[334]	training's l1: 1103.3	valid_1's l1: 1122.49
[335]	training's l1: 1103.25	valid_1's l1: 1122.45
[336]	training's l1: 1103.17	valid_1's l1: 1122.41
[337]	training's l1: 1103.05	valid_1's l1: 1122.28
[338]	training's l1: 1102.96	valid_1's l1: 1122.25
[339]	training's l1: 1102.95	valid_1's l1: 1122.25
[340]	training's l1: 1102.9	valid_1's l1: 1122.22
[341]	training's l1: 1102	valid_1's l1: 1121.37
[342]	training's l1: 1101.9	valid_1's l1: 1121.31
[343]	training's l1: 1101.88	valid_1's l1: 1121.31
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[344]	training's l1: 1101.73	valid_1's l1: 1121.18
[345]	training's l1: 1101.68	valid_1's l1: 1121.16
[346]	training's l1: 1101.47	valid_1's l1: 1121.08
[347]	training's l1: 1101.43	valid_1's l1: 1121.03
[348]	training's l1: 1101.36	valid_1's l1: 1121.03
[349]	training'

[505]	training's l1: 1083.54	valid_1's l1: 1110.54
[506]	training's l1: 1083.5	valid_1's l1: 1110.51
[507]	training's l1: 1083.39	valid_1's l1: 1110.44
[508]	training's l1: 1083.33	valid_1's l1: 1110.4
[509]	training's l1: 1081.74	valid_1's l1: 1108.24
[510]	training's l1: 1081.68	valid_1's l1: 1108.22
[511]	training's l1: 1081.54	valid_1's l1: 1108.12
[512]	training's l1: 1081.52	valid_1's l1: 1108.09
[513]	training's l1: 1081.08	valid_1's l1: 1107.66
[514]	training's l1: 1081.06	valid_1's l1: 1107.66
[515]	training's l1: 1081.04	valid_1's l1: 1107.67
[516]	training's l1: 1081	valid_1's l1: 1107.64
[517]	training's l1: 1080.96	valid_1's l1: 1107.63
[518]	training's l1: 1080.87	valid_1's l1: 1107.58
[519]	training's l1: 1080.86	valid_1's l1: 1107.57
[520]	training's l1: 1080.85	valid_1's l1: 1107.57
[521]	training's l1: 1080.73	valid_1's l1: 1107.53
[522]	training's l1: 1078.42	valid_1's l1: 1106
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[523]	training'

[665]	training's l1: 1066.15	valid_1's l1: 1097.59
[666]	training's l1: 1065.9	valid_1's l1: 1097.39
[667]	training's l1: 1065.86	valid_1's l1: 1097.35
[668]	training's l1: 1065.83	valid_1's l1: 1097.33
[669]	training's l1: 1065.81	valid_1's l1: 1097.35
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[670]	training's l1: 1065.78	valid_1's l1: 1097.35
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[671]	training's l1: 1065.75	valid_1's l1: 1097.36
[672]	training's l1: 1065.72	valid_1's l1: 1097.34
[673]	training's l1: 1065.7	valid_1's l1: 1097.33
[674]	training's l1: 1065.69	valid_1's l1: 1097.33
[675]	training's l1: 1065.67	valid_1's l1: 1097.34
[676]	training's l1: 1065.38	valid_1's l1: 1096.98
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[677]	training's l1: 1065.34	valid_1's l1: 1096.96
[678]	training's l1: 1065.26	valid_1's l1: 1096.91
[LightGBM] [Warning] No further splits with positive gain, best

...

[896]	training's l1: 710.999	valid_1's l1: 936.511
[897]	training's l1: 711.027	valid_1's l1: 936.37
[898]	training's l1: 710.901	valid_1's l1: 936.312
[899]	training's l1: 710.869	valid_1's l1: 936.298
[900]	training's l1: 710.841	valid_1's l1: 936.287
[901]	training's l1: 710.584	valid_1's l1: 936.116
[902]	training's l1: 710.396	valid_1's l1: 936.016
[903]	training's l1: 710.328	valid_1's l1: 935.976
[904]	training's l1: 710.243	valid_1's l1: 935.914
[905]	training's l1: 710.162	valid_1's l1: 935.841
[906]	training's l1: 710.11	valid_1's l1: 935.819
[907]	training's l1: 710.013	valid_1's l1: 935.738
[908]	training's l1: 709.849	valid_1's l1: 935.62
[909]	training's l1: 709.72	valid_1's l1: 935.557
[910]	training's l1: 709.667	valid_1's l1: 935.518
[911]	training's l1: 709.588	valid_1's l1: 935.456
[912]	training's l1: 709.468	valid_1's l1: 935.375
[913]	training's l1: 709.378	valid_1's l1: 935.296
[914]	training's l1: 709.335	valid_1's l1: 935.258
[915]	training's l1: 709.242	valid_

[1057]	training's l1: 702.118	valid_1's l1: 930.298
[1058]	training's l1: 702.085	valid_1's l1: 930.266
[1059]	training's l1: 701.99	valid_1's l1: 930.217
[1060]	training's l1: 702.023	valid_1's l1: 930.205
[1061]	training's l1: 701.998	valid_1's l1: 930.186
[1062]	training's l1: 701.926	valid_1's l1: 930.124
[1063]	training's l1: 701.848	valid_1's l1: 930.027
[1064]	training's l1: 701.857	valid_1's l1: 930.005
[1065]	training's l1: 701.826	valid_1's l1: 929.984
[1066]	training's l1: 702.037	valid_1's l1: 930.219
[1067]	training's l1: 701.993	valid_1's l1: 930.196
[1068]	training's l1: 701.967	valid_1's l1: 930.174
[1069]	training's l1: 701.769	valid_1's l1: 930.085
[1070]	training's l1: 701.717	valid_1's l1: 930.039
[1071]	training's l1: 701.679	valid_1's l1: 929.989
[1072]	training's l1: 701.594	valid_1's l1: 929.897
[1073]	training's l1: 701.566	valid_1's l1: 929.882
[1074]	training's l1: 701.503	valid_1's l1: 929.827
[1075]	training's l1: 701.445	valid_1's l1: 929.786
[1076]	traini

[1230]	training's l1: 695.025	valid_1's l1: 926.062
[1231]	training's l1: 695.006	valid_1's l1: 926.043
[1232]	training's l1: 694.974	valid_1's l1: 926.023
[1233]	training's l1: 695.102	valid_1's l1: 926.048
[1234]	training's l1: 695.105	valid_1's l1: 926.036
[1235]	training's l1: 695.062	valid_1's l1: 926.004
[1236]	training's l1: 695.058	valid_1's l1: 926.009
[1237]	training's l1: 695.041	valid_1's l1: 926.001
[1238]	training's l1: 695.019	valid_1's l1: 925.982
[1239]	training's l1: 695.018	valid_1's l1: 925.973
[1240]	training's l1: 694.997	valid_1's l1: 925.971
[1241]	training's l1: 694.983	valid_1's l1: 925.97
[1242]	training's l1: 694.974	valid_1's l1: 925.978
[1243]	training's l1: 694.953	valid_1's l1: 925.969
[1244]	training's l1: 694.924	valid_1's l1: 925.947
[1245]	training's l1: 694.979	valid_1's l1: 925.955
[1246]	training's l1: 694.8	valid_1's l1: 925.801
[1247]	training's l1: 694.838	valid_1's l1: 925.855
[1248]	training's l1: 694.79	valid_1's l1: 925.82
[1249]	training's

[1405]	training's l1: 690.304	valid_1's l1: 923.475
[1406]	training's l1: 690.278	valid_1's l1: 923.449
[1407]	training's l1: 690.253	valid_1's l1: 923.431
[1408]	training's l1: 690.247	valid_1's l1: 923.43
[1409]	training's l1: 690.222	valid_1's l1: 923.418
[1410]	training's l1: 690.204	valid_1's l1: 923.4
[1411]	training's l1: 690.168	valid_1's l1: 923.377
[1412]	training's l1: 690.153	valid_1's l1: 923.365
[1413]	training's l1: 690.082	valid_1's l1: 923.286
[1414]	training's l1: 690.072	valid_1's l1: 923.278
[1415]	training's l1: 690.059	valid_1's l1: 923.275
[1416]	training's l1: 690.63	valid_1's l1: 923.882
[1417]	training's l1: 690.617	valid_1's l1: 923.873
[1418]	training's l1: 690.613	valid_1's l1: 923.869
[1419]	training's l1: 690.591	valid_1's l1: 923.851
[1420]	training's l1: 690.562	valid_1's l1: 923.794
[1421]	training's l1: 690.556	valid_1's l1: 923.79
[1422]	training's l1: 690.545	valid_1's l1: 923.782
[1423]	training's l1: 690.52	valid_1's l1: 923.76
[1424]	training's l

,feature,gain,split
0,month_indicator,1342.185157,28
16,sales_univ_b1_6mo,730.141197,13
8,sales_univ_brand_1mo,675.276480,37
147,sales_region_b3market_trend_6mo,643.637265,42
110,sales_univ_brand_2mo,605.682720,29
128,sales_region_b3_6mo,591.527781,47
121,sales_univ_brand_cummax,413.154075,25
120,sales_univ_brand_cumsum,401.572770,15
136,sales_region_b3_cummin,290.170353,31
1,sales_univ_b1_1mo,277.766572,4


### 3.5 Prediction storage

In [6]:
submission = master[(master.sales.isna()) & (master.month >= '2020-07')].copy()
submission['sales'] = lgb_quantiles[0.5].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['lower'] = lgb_quantiles[0.1].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales']))
submission['upper'] = lgb_quantiles[0.9].predict(submission.drop(columns = ['month', 'region', 'brand', 'sales', 'lower']))
submission = submission[['month', 'region', 'brand', 'sales', 'lower', 'upper']]

submission.loc[submission.sales < 0, 'sales'] = 0
submission.loc[submission.lower < 0, 'lower'] = 0
submission.loc[submission.upper < 0, 'upper'] = 0
submission.head()

,month,region,brand,sales,lower,upper
2714,2020-07,region_151,brand_1,51.320691,0.0,473.995188
2715,2020-07,region_151,brand_2,51.320691,0.0,473.995188
2716,2020-07,region_152,brand_1,33.984140,0.0,809.872751
2717,2020-07,region_152,brand_2,33.984140,0.0,809.872751
2718,2020-07,region_153,brand_1,115.220470,0.0,923.674564


In [7]:
submission.to_csv(os.path.join(results_path, 'submission10_team46.csv'), index = False)

In [8]:
(submission['upper'] - submission['lower']).mean()

# Submission 2 difference - 1815.437323083508
# Submission 3 difference - 1613.6602449420075
# Submission 4 difference - 1145.0976140382802
# Submission 5 difference - 1132.1785576554485
#1222.15

1800.158482545283

In [8]:
preds_train = pd.DataFrame({'month': master[(master.sales.isna()==False) & (master.month >= '2020-07')].month,
                            'region': master[(master.sales.isna()==False) & (master.month >= '2020-07')].region,
                            'brand': master[(master.sales.isna()==False) & (master.month >= '2020-07')].brand,
                            'predictions': lgb_quantiles[0.5].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'lower': lgb_quantiles[0.1].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'upper': lgb_quantiles[0.9].predict(master[(master.sales.isna()==False) &
                                                                             (master.month >= '2020-07')].drop(columns = ['month', 'region', 'brand', 'sales'])),
                            'sales': master[(master.sales.isna()==False) & (master.month >= '2020-07')].sales
                           })

preds_train.loc[preds_train.predictions < 0, 'predictions'] = 0
preds_train.loc[preds_train.lower < 0, 'lower'] = 0
preds_train.loc[preds_train.upper < 0, 'upper'] = 0

width = abs(preds_train['upper'] - preds_train['lower'])
below = (preds_train['lower'] - preds_train['sales'])*(preds_train['lower'] > preds_train['sales'])
above = (preds_train['sales'] - preds_train['upper'])*(preds_train['upper'] < preds_train['sales'])
outside = (2/0.2)*(below + above)

In [9]:
preds_train['outside'] = (preds_train.predictions < preds_train.lower)  | (preds_train.predictions > preds_train.upper)
preds_train.outside.mean()

0.11660359508041628

In [9]:
width.mean()
# 0.25 988.73
# 0.2  1090.40
# 0.1 1827.48

1090.40306718365

In [11]:
outside.mean()
# 0.25 1276.75
# 0.2  632.81
# 0.1 209.00

632.8174561004448

In [12]:
(width + outside).mean()
# 0.25 2265.492
# 0.2  1723.220
# 0.1 2036.48

1723.220523284095